In [1]:
import pandas as pd
import os
from tqdm import tqdm
import urllib.parse
import urllib.request
from uniprotRetrieve import uniprotRetrieve
import numpy as np
import matplotlib.pyplot as plt
import gc

### 1.1.1 Get twins (Bacteria, any evidence)

In previous analysis, 
a twins directory was generated.
In this directory, 
cytoplasm / secretome twims were listed in a table for each of the different organisms.

In [2]:
DIR="../2020-04-06.FindTwinsAnyEvidenceBacteria/3.3.twins/"
TWINS_COMBINED=list()                    # Empty list that will be appended in a loop
for FILE in tqdm([F for F in os.listdir(DIR) if F.endswith(".tab")]):
    """
    Loop through all the files inside the twins directory.
    Each file corresponds to a bacterial organism and inside is a list of twins.
    """
    ORGANISM=FILE.replace(".tab","")     # Extract organism from file
    FILE_PATH="{}{}".format(DIR,FILE)    # Get full path of FILE
    F = open(FILE_PATH)
    F.readline()                         # First line is header and can be skipped
    for LINE in F:
        CYTOPLASM_PROTEIN, PERIPLASM_PROTEIN = LINE.strip().split("\t")
        TWINS_COMBINED.append((CYTOPLASM_PROTEIN, PERIPLASM_PROTEIN, ORGANISM))

100%|██████████| 776/776 [00:00<00:00, 1838.43it/s]


In [3]:
TWINS = pd.DataFrame(TWINS_COMBINED, columns=["Cytoplasm", "Periplasm", "Organism"])
TWINS

,Cytoplasm,Periplasm,Organism
0,A0A5I0AHK7,A0A5I0AM00,Salmonella_enterica_subsp._enterica_serovar_Vi...
1,E6BT71,E6BLU6,Escherichia_coli_MS_85-1
2,A0A5I5GAJ9,A0A5I5G9G4,Salmonella_enterica_subsp._enterica_serovar_Ne...
3,D2AA92,D2ADI4,Shigella_flexneri_serotype_X_strain_2002017
4,A0A5H5RFJ5,A0A5H6V778,Salmonella_enterica_subsp._enterica_serovar_Bu...
...,...,...,...
31160,A0A3T3G9X3,A0A3T3G349,Salmonella_enterica_subsp._enterica_serovar_Br...
31161,A0A080J5A2,A0A080J791,Escherichia_coli_1-250-04_S3_C2
31162,A0A5I1I569,A0A5H7PWL6,Salmonella_enterica_subsp._enterica_serovar_Su...
31163,D8E624,D8E319,Escherichia_coli_MS_119-7


There are 31.165 possible twins.
In this groups there are related proteins though.
It could be that cytoplasmic protein A and B are closely related. 
If they form a Twin with C,D en E, this will result in 6 pairs:
(AC, AD, AE, BC, BD, BE).

In [4]:
CYTOPLASM_UNIQUE = TWINS["Cytoplasm"].unique()
len(CYTOPLASM_UNIQUE)

1325

There 1325 unique cytoplasmic proteins.

In [5]:
PERIPLASM_UNIQUE = TWINS["Periplasm"].unique()
len(PERIPLASM_UNIQUE)

1902

There are 1902 unique periplasmic proteins.

### 1.1.2 Look up signal peptide positions for periplasmic proteins

To compare amino acids of periplasmic and cytoplasmic proteins, 
the signal peptide should be cut of.
For that it is necessary to now where the start and end position of the signal peptide.
To achieve this goal in a quick and efficient manner,
the uniprot mapper API will be used to map the uniprot Ids to themselves.
In the protein a group ID is generated (which is remembered by uniprot for one week.
With this group Id, a usual uniprot search query can be performed to retrieve the signal peptide information.

In [6]:
"""Put parameters that describe what you want to request from the mapper 
in a dictionary."""
PARAMS={
        "from":"ACC",
        "to":"ACC",
        "format":"tab",
        "columns":"id"
        }
PARAMS["query"]="\n".join(PERIPLASM_UNIQUE)

"""Encode parameters into a form that can be passed by urllib (use UTF-8 encoding)"""
DATA = urllib.parse.urlencode(PARAMS)
DATA = DATA.encode('utf-8')

"""Request mapping information from Uniprot Mapping API"""
URL="https://www.uniprot.org/uploadlists/"
REQ = urllib.request.Request(URL, DATA)
with urllib.request.urlopen(REQ) as F:
    # Only first line needs to be read to get group ID
    GROUP_ID = str(F.readline(), 'utf-8').strip().split("\t")[-1]
GROUP_ID

'yourlist:M20200512216DA2B77BFBD2E6699CA9B6D1C41EB287BC8CC'

In [7]:
# Use Uniprot search query API to retrieve signal peptide information
FILE_NAME="periplasm_signal_peptides.tab"
QUERY=GROUP_ID
FORMAT="tab"
COLUMNS="id,feature(SIGNAL)"
uniprotRetrieve(FILE_NAME,query=QUERY,format=FORMAT, columns=COLUMNS)

'periplasm_signal_peptides.tab'

In [8]:
# Map signal peptide info back to twins
FILE_NAME="periplasm_signal_peptides.tab"

SIGNAL_PEP_START_END=list()

F = open(FILE_NAME)
for LINE in F:
    ID,SIGNAL_PEP_INFO=LINE.split("\t")
    if SIGNAL_PEP_INFO.startswith("SIGNAL"):
        START_END=SIGNAL_PEP_INFO.replace(";","").split(" ")[1]
        START,END= [int(ELM) for ELM in START_END.split("..")]
        SIGNAL_PEP_START_END.append((ID,START,END))
F.close()
    
SIGNAL_PEPTIDES = pd.DataFrame(SIGNAL_PEP_START_END, columns=["Periplasm","SP_start","SP_end"])
SIGNAL_PEPTIDES

,Periplasm,SP_start,SP_end
0,P00805,1,22
1,Q59635,1,30
2,Q72EC8,1,28
3,P25718,1,17
4,Q7M827,1,33
...,...,...,...
1764,A0A5C6XDI0,1,36
1765,A0A1C4AWP6,1,26
1766,A0A2U8Y9B8,1,30
1767,A0A0K9T8F9,1,30


In [9]:
# Add information to TWINS dataframe and write to file
TWINS_WITH_SP = pd.merge(TWINS, SIGNAL_PEPTIDES, on="Periplasm")
FILE_NAME="twins_with_org_sp.tab"
TWINS_WITH_SP.to_csv(FILE_NAME,sep="\t", index=False)

### 1.1.3 Retrieve MSA

In [10]:
def readMSA(file):
    """
    Reads MSA file in fasta format and return dictionary:
        key: protein ID
        value: gabbed sequence
    """
    with open(file) as f:
        lines = [line.strip() for line in f.readlines()]
    msa=dict()
    for line in lines:
        if line.startswith(">"):
            # generate new key for new proteinId
            Id=line.replace(">","")
            msa[Id]=""
        else:
            # add sequence to entry
            msa[Id]+=line
    return msa

In [11]:
"""Retrieve all the MSA sequences and group them by main protein"""
DIR="../2020-04-06.FindTwinsAnyEvidenceBacteria/4.7.clustalOmega/"
ALL_MSA=dict()
for FILE in tqdm([F for F in os.listdir(DIR) if F.endswith(".fasta")]):
    FILE_PATH="{}{}".format(DIR,FILE)
    ID=FILE.replace(".MSA.fasta","")
    ALL_MSA[ID]=readMSA(FILE_PATH)

100%|██████████| 386/386 [00:00<00:00, 1092.51it/s]


In [12]:
# Split into cytoplasmic and periplasmic groups
CYTOPLASMIC_MSA = {ID:VALUE for ID,VALUE in ALL_MSA.items() if ID in CYTOPLASM_UNIQUE}
PERIPLASMIC_MSA = {ID:VALUE for ID,VALUE in ALL_MSA.items() if ID in PERIPLASM_UNIQUE}
len(CYTOPLASMIC_MSA), len(PERIPLASMIC_MSA)

(187, 199)

### 1.1.4 Retrieve Predictions

In [13]:
def readPredictions(file):
    """
    Reads prediction file and returns dictionary
        key: protein ID
        value: list of predictions
    """
    with open(file) as f:
        # Strip lines and leave out empty lines
        lines = [line.strip() for line in f.readlines() if len(line.strip())>0]
    predictions=dict()
    for line in lines:
        if line.startswith("* for "):
            # Extract Id from header and assign to dictionary
            Id=line.replace("* for ","").replace(" ","").replace("*","")
            predictions[Id]=list()
        elif line.startswith("*"):
            # Skip lines of the headers
            pass
        else:
            residue, pred_raw = line.replace(" ","").split("\t")
            pred=float(pred_raw)
            predictions[Id].append((residue,pred))
    return predictions

In [14]:
"""Retrieve all the MSA predictions and group them by main protein"""
DIR="../2020-04-20.efoldMinePrediction/AnyEvidenceBacteria"
PREDICTIONS_ALL=dict()
for ID in tqdm(os.listdir(DIR)):
    SUBDIR="{}/{}".format(DIR,ID)
    PREDICTIONS_ALL[ID]=dict()
    for FILE in [F for F in os.listdir(SUBDIR) if F!=".ipynb_checkpoints"]:
        FILE_PATH="{}/{}".format(SUBDIR,FILE)
        PRED_TYPE=FILE.replace(".pred","").split("_")[-1]
        PREDICTIONS_ALL[ID][PRED_TYPE]=readPredictions(FILE_PATH)

100%|██████████| 386/386 [01:52<00:00,  3.42it/s]


In [15]:
# Split into cytoplasmic and periplasmic groups
# This filles up the memory very close to 8GB
CYTOPLASMIC_PREDICTIONS = {ID:VALUE for ID,VALUE in tqdm(PREDICTIONS_ALL.items()) if ID in CYTOPLASM_UNIQUE}
PERIPLASMIC_PREDICTIONS = {ID:VALUE for ID,VALUE in tqdm(PREDICTIONS_ALL.items()) if ID in PERIPLASM_UNIQUE}
len(CYTOPLASMIC_PREDICTIONS), len(PERIPLASMIC_PREDICTIONS)

100%|██████████| 386/386 [00:00<00:00, 6552.54it/s]


(187, 199)

### 1.1.5 Map Predictions to MSA

In [16]:
def mapPredictions(predictions, msa):
    """
    Uses predictions dictionary and msa dictionary as input and returns mapped predictions dictionary:
        key:protein ID
        value: mapped prediction
    """
    def map(gabbedSequence, predictionSequence, gab="-"):
        """
        Map one gabbed MSA sequence to a prediction sequence.
            Returns mapped prediction sequence with np.nan values at the gabs
        """
        if len(gabbedSequence.replace(gab,""))!=len(predictionSequence):
            print("There is an unequal amount of predictions and residues in the MSA")
            raise
        # dummy so that there is no out of range ERROR
        predictionSequence=predictionSequence[:]+[("dummy","dummy")]
        """This list will be filled up:
            * a prediction value if there is a residue at the position
            * np.nan value if there is no residue """
        gabbedPredictionSequence=list()
        """Integer of where we are in prediction Sequence
        Everytime we encounter a residue in the MSA sequence, this value will go up with one"""
        iPredictionSequence=0
        for res in gabbedSequence:
            predictionRes=predictionSequence[iPredictionSequence][0]
            predictionValue=predictionSequence[iPredictionSequence][1]
            if res==gab:
                """Gabs are mapped to np.nan values"""
                gabbedPredictionSequence.append(np.nan)
            elif res!=predictionRes:
                print("residue of prediction and residue in MSA should be the same")
                raise
            else:
                """Residues are mapped to there prediction"""
                gabbedPredictionSequence.append(predictionValue)
                iPredictionSequence+=1
        return gabbedPredictionSequence
    
    """Identifieres in MSA and predictions should be the same:
        Mapping will only happen for those Identifiers that occur in both the MSA and prediction set"""
    msaIds = set(msa.keys())
    mappedPredictions=dict()
    for predictionType in predictions.keys():
        predictionIds=set(predictions[predictionType].keys())
        Ids=msaIds&predictionIds
        for Id in Ids:
            if Id not in mappedPredictions.keys():
                mappedPredictions[Id]=dict()
            msaSequence=msa[Id]
            predictionSequence=predictions[predictionType][Id]
            mappedPredictions[Id]["msa"]=msaSequence
            mappedPredictions[Id][predictionType]=map(msaSequence,predictionSequence)
    
    '''
    for predictionType in predictions[id1].keys():
        predictionIds = set(predictions[predictionType].keys())
        print(predictionIds)
        msaIds = set(msa.keys())
        if msaIds!=predictionIds:
            print("""provided protein IDs for MSA not the same as IDs for predictions, 
                  taking the intersection of both sets""")
            ids=sorted(predictionIds&msaIds)
        else:
            ids=sorted(msaIds)
        mappedPredictions=dict()
        for Id in ids:
            msaSequence=msa[Id]
            predictionSequence=predictions[predictionType][Id]
            mappedPredictions[Id]=dict()
            mappedPredictions[Id]["msa"]=msaSequence
            mappedPredictions[Id][predictionType]=map(msaSequence,predictionSequence)
    '''
    return mappedPredictions

In [17]:
"""Map all the predictions to their MSA
Because it is not possible to hold everythin and all previous in memory at the same time.
it will be written sequentially to files"""
def writeMappedMsaPrediction(fileName, mappedMsaPrediction):
    """
    Takes as input dictionary with mapped MSA predictions and filename.
    Writes to file.
    """
    output=open(fileName,"w")
    for Id,values in mappedMsaPrediction.items():
        output.write(">{}\n".format(Id))
        for predictionKey,predictionValue in values.items():
            toWrite="{}\t{}\n".format(predictionKey," ".join([str(elm) for elm in list(predictionValue)]))
            output.write(toWrite)
    output.close()

OUTPUT_DIR="mappedPredictions"    
for ID in tqdm(ALL_MSA.keys()):
    MAPPED_MSA_PREDICTION = mapPredictions(PREDICTIONS_ALL[ID], ALL_MSA[ID])
    OUTPUT_FILE="{}/{}_mapped_predictions.txt".format(OUTPUT_DIR,ID)
    writeMappedMsaPrediction(OUTPUT_FILE,MAPPED_MSA_PREDICTION)

100%|██████████| 386/386 [01:05<00:00,  5.86it/s]


### 1.1.6 Sort predictions per Twin

To avoid redundancy in the dataset,
only one Twin was taken if multiple were available for similar proteins.

In [40]:
CYTOPLASM_WITH_MSA=pd.DataFrame(CYTOPLASMIC_MSA.keys(),columns=["Cytoplasm"])
PERIPLASM_WITH_MSA=pd.DataFrame(PERIPLASMIC_MSA.keys(),columns=["Periplasm"])
TWINS_UNIQUE_WITH_MSA = TWINS_WITH_SP\
.merge(CYTOPLASM_WITH_MSA,on="Cytoplasm")\
.merge(PERIPLASM_WITH_MSA,on="Periplasm")\
.groupby(["Cytoplasm"],as_index=False)\
.first()\
.groupby(["Periplasm"],as_index=False)\
.first()

In [42]:
TWINS_UNIQUE_WITH_MSA.to_csv("twins_unique_with_msa.tab",sep="\t")